# Some traffic prediction
variation by Artem Ostrovsky to the original notebook created by Anvar Kurmukov,
updated by Bogdan Kirillov, Hekmat Taherinejad, Satyarth Mishra Sharma

---

Try to implement your knowledge in Python functions, classes and debugging:

1. Make DataAccessor.py -> class that creates handy interface to all your data
2. Make Features.py -> class that creates some new artifitial features
3. Add try/exept
4. Run the pipeline using pdb

---

Some links:

pdb https://habr.com/ru/post/104086/, https://pythonim.ru/moduli/pdb-python

classes https://www.w3schools.com/python/python_classes.asp

---

We are using a public dataset compiling weather information and traffic data continuously monitored in the Twin Cities, Minnesota from 2012 to 2018. The dataset page can be found [here](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume).

You need to download `Metro_Interstate_Traffic_Volume.csv` and place it in the same directory as this notebook.


In [1]:
import numpy as np
import pandas as pd

# 1. Loading data

As always in Data Science you are starting with making nice cup of tea (or coffee). Your next move is to load the data:

- Start with loading `Metro_Interstate_Traffic_Volume.csv` file using `pd.read_csv()` function.
- You may also want to increase maximal displayed pandas columns: set `pd.options.display.max_columns` to 30
- Print top 10 observations in the table. `.head()`
- Print last 10 observations in the table. `.tail()`
- Print all the data columns names using method `.columns`
- Print data size (number of rows and columns). This is the `.shape` of the data.

*Almost* every python has a `head` and a `tail` just as DataFrames do.

If you are using Google Colab, you can upload the file in the cell below. If you are NOT using Colab, set COLAB_P in the cell below to False.

In [2]:
# Load the data
data = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')
data

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0
1,None,289.36,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0
2,None,289.58,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0
3,None,290.13,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0
4,None,291.14,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0
...,...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0
48200,None,282.76,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0
48201,None,282.73,0.0,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0
48202,None,282.09,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 22:00:00,1450.0


In [3]:
# Observe top 10 observations (int)
data.head(10)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0
1,None,289.36,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0
2,None,289.58,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0
3,None,290.13,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0
4,None,291.14,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0
5,None,291.72,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 14:00:00,5181.0
6,None,293.17,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 15:00:00,5584.0
7,None,293.86,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 16:00:00,6015.0
8,None,294.14,0.0,0.0,20.0,Clouds,few clouds,2012-10-02 17:00:00,5791.0
9,None,293.10,0.0,0.0,20.0,Clouds,few clouds,2012-10-02 18:00:00,4770.0


In [4]:
# Observe last 10 observations (int)
data.tail(10)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
48194,None,283.84,0.00,0.0,75.0,Rain,proximity shower rain,2018-09-30 15:00:00,4302.0
48195,None,283.84,0.00,0.0,75.0,Drizzle,light intensity drizzle,2018-09-30 15:00:00,4302.0
48196,None,284.38,0.00,0.0,75.0,Rain,light rain,2018-09-30 16:00:00,4283.0
48197,None,284.79,0.00,0.0,75.0,Clouds,broken clouds,2018-09-30 17:00:00,4132.0
48198,None,284.20,0.25,0.0,75.0,Rain,light rain,2018-09-30 18:00:00,3947.0
48199,None,283.45,0.00,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0
48200,None,282.76,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0
48201,None,282.73,0.00,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0
48202,None,282.09,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 22:00:00,1450.0
48203,None,282.12,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 23:00:00,954.0


In [5]:
# Print all the columns/features names (int)
data.columns.tolist()

['holiday',
 'temp',
 'rain_1h',
 'snow_1h',
 'clouds_all',
 'weather_main',
 'weather_description',
 'date_time',
 'traffic_volume']

In [6]:
# Q1.1 How many columns end with a vowel?
vowel = 'eyuioa'
print('Starts with vowel:', sum([data.columns.str.startswith(s).sum() for s in vowel]))

# Q1.2 How many columns start with a vowel?
vowel = 'eyuioa'
print('Ends with vowel:', sum([data.columns.str.endswith(s).sum() for s in vowel]))

# Q1.3 Which columns are associated with the condition of weather?
print(list(set(data.columns) - {'holiday', 'date_time', 'traffic_volume'}))

# Q1.4 How many columns have `th` in their names?
print('Contains "th":', data.columns.str.contains('th').sum())

Starts with vowel: 0
Ends with vowel: 3
['temp', 'weather_main', 'snow_1h', 'rain_1h', 'weather_description', 'clouds_all']
Contains "th": 2


In [7]:
# Print data size (int)
print('Data shape:', data.shape)

# Q2.1 How many observations are in the data?
print('Observations:', data.shape[0])

# Q2.2 How many features are in the data?
print('Features:', data.shape[1])

Data shape: (48204, 9)
Observations: 48204
Features: 9


# 2. Basic data exploration

Lets do some basics:

`.count()` number of not NaN's in every column.
    
Is there any missing values in the data?     
Count number of unique values in every column .nunique().    
What does this tells you about the features, which are most likely categorical and which are most likely numerical?    
Use pandas `.describe()` to display basic statistic about the data.   
Use pandas `.value_counts()` to count number of unique values in a specific column.   
Use pandas `.min()`, `.max()`, `.mean()`, `.std()` to display specific statistics about the data.    
Use pandas `.dtypes` field to display data types in columns. 
Hint You could use `.sort_index()` or `.sort_values()` to sort the result of `.value_counts()`


In [8]:
# Display number of not NaN's in every column (int)
# print(data.count())
# print('------------------------')

# Q3.1 How many NA values are in the `clouds_all` column?
print('NaNs in `clouds_all`:', data.clouds_all.isna().sum()) #OR data.shape[0] - data.clouds_all.count()

# Q3.2 How many NA values are in the `temp` column?
print('NaNs in `temp`:', data.temp.isna().sum())

# Q3.3 How many NA values are in the `rain_1h` column?
print('NaNs in `rain_1h`:', data.rain_1h.isna().sum())

# Q3.4 How many NA values are in the `snow_1h` column?
print('NaNs in `snow_1h`:', data.snow_1h.isna().sum())

# Q3.5 How many explicit NA values are in the `traffic_volume` column?
print('NaNs in `traffic_volume`:', data.traffic_volume.isna().sum())
print('NaNs (including explicit) in `traffic_volume`:', ((data.traffic_volume.isna())|(data.traffic_volume == 0)).sum())

NaNs in `clouds_all`: 3
NaNs in `temp`: 1
NaNs in `rain_1h`: 1
NaNs in `snow_1h`: 0
NaNs in `traffic_volume`: 5
NaNs (including explicit) in `traffic_volume`: 7


In [9]:
# Now drop rows with NaN with `.dropna`. Remeber to either reassign your dataframe or provide `inplace=True` argument.
data.dropna(axis=0, inplace=True)

In [10]:
# Display basic data statistics using .describe()
data.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,48190.000000,48190.000000,48190.000000,48190.000000,48190.000000
mean,281.201366,0.334356,0.000222,49.369267,3259.859079
std,13.337406,44.795638,0.008169,39.016127,1986.972809
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.160000,0.000000,0.000000,1.000000,1192.250000
50%,282.440000,0.000000,0.000000,64.000000,3380.000000
75%,291.800000,0.000000,0.000000,90.000000,4933.000000
max,310.070000,9831.300000,0.510000,100.000000,7280.000000


In [11]:
# Count number of unique values in every column (int)
# print(data.nunique())
# print('------------------------')

# Q4.1 How many unique values are in the `clouds_all` column?
print('Uniques in `clouds_all`:', data.clouds_all.nunique())

# Q4.2 How many unique values are in the `weather_main` column?
print('Uniques in `weather_main`:', data.weather_main.nunique())

# Q4.3 How many unique values are in the `weather_description` column?
print('Uniques in `weather_description`:', data.weather_description.nunique())

# Q4.4 How many unique values are in the `snow_1h` column?
print('Uniques in `snow_1h`:', data.snow_1h.nunique())

# Q4.5 How many unique values are in the `rain_1h` column?
print('Uniques in `rain_1h`:', data.rain_1h.nunique())

print('------------------------')
for column in ['clouds_all', 'weather_main', 'weather_description', 'snow_1h', 'rain_1h']:
    print(f'Uniques in `{column}`:', data[column].nunique())


Uniques in `clouds_all`: 60
Uniques in `weather_main`: 11
Uniques in `weather_description`: 38
Uniques in `snow_1h`: 12
Uniques in `rain_1h`: 372
------------------------
Uniques in `clouds_all`: 60
Uniques in `weather_main`: 11
Uniques in `weather_description`: 38
Uniques in `snow_1h`: 12
Uniques in `rain_1h`: 372


In [12]:
# Count frequency of the values in different columns (list of ints in ascending order)
# You could select a column using same syntax as for selecting a key from a dictionary: `data[colname]`
# numpy's `unique` function can be useful for this task

# for column in data.columns:
#     print(data[column].value_counts())
# print('------------------------')

# Q5.1 For every unique `weather_main` value give its number of occurences.
print(data.weather_main.value_counts())
print('---------------------------------------------')

# Q5.2 For every unique `weather_description` value give its number of occurences.
print(data.weather_description.value_counts())

Clouds          15159
Clear           13384
Mist             5950
Rain             5671
Snow             2876
Drizzle          1821
Haze             1359
Thunderstorm     1034
Fog               912
Smoke              20
Squall              4
Name: weather_main, dtype: int64
---------------------------------------------
sky is clear                           11658
mist                                    5950
overcast clouds                         5081
broken clouds                           4665
scattered clouds                        3458
light rain                              3371
few clouds                              1955
light snow                              1946
Sky is Clear                            1726
moderate rain                           1664
haze                                    1359
light intensity drizzle                 1100
fog                                      912
proximity thunderstorm                   673
drizzle                                  651
heav

In [13]:
# Display some column statistics (list of floats, rounded up to 3 digits, e.g. 1.234)

# Q6.1 What are the max, min, mean and the std of the `traffic_volume` column?
# Q6.2 What are the max, min, mean and the std of the `clouds_all` column?
# Q6.3 What are the max, min, mean and the std of the `temp` column?
# Q6.4 What are the max, min, mean and the std of the `rain_1h` column?
# Q6.5 What are the max, min, mean and the std of the `snow_1h` column?

for column in ['traffic_volume', 'clouds_all', 'temp', 'rain_1h', 'snow_1h']:
    print(f'Statistics in `{column}`:')
    print('max:', data[column].max().round(3))
    print('min:', data[column].min().round(3))
    print('mean:', data[column].mean().round(3))
    print('std:', data[column].std().round(3))
    print('---------------------')

# for column in ['traffic_volume', 'clouds_all', 'temp', 'rain_1h', 'snow_1h']:
#     print(f'Statistics in `{column}`')
#     for func in ['max', 'min', 'mean', 'std']:
#         print(data[column].apply(func).round(3))
#     print('---------------------')

Statistics in `traffic_volume`:
max: 7280.0
min: 0.0
mean: 3259.859
std: 1986.973
---------------------
Statistics in `clouds_all`:
max: 100.0
min: 0.0
mean: 49.369
std: 39.016
---------------------
Statistics in `temp`:
max: 310.07
min: 0.0
mean: 281.201
std: 13.337
---------------------
Statistics in `rain_1h`:
max: 9831.3
min: 0.0
mean: 0.334
std: 44.796
---------------------
Statistics in `snow_1h`:
max: 0.51
min: 0.0
mean: 0.0
std: 0.008
---------------------


In [14]:
# Display data types of all columns (int)

# Q7.1 How many columns have `object` data type?
print('Number of `object`:', (data.dtypes == 'object').sum())

# Q7.2 How many columns have `int64` data type?
print('Number of `int64`:', (data.dtypes == 'int64').sum())

# Q7.3 How many columns have `float64` data type?
print('Number of `float64`:', (data.dtypes == 'float64').sum())

# Q7.4 What are the columns with dtype == `float64`?
print('Columns with dtype: `float64`:', data.dtypes[data.dtypes == 'float64'].keys().tolist())

# Q7.5 What are the columns with dtype == `int64`?
print('Columns with dtype: `int64`:', data.dtypes[data.dtypes == 'int64'].keys().tolist())

Number of `object`: 4
Number of `int64`: 0
Number of `float64`: 5
Columns with dtype: `float64`: ['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']
Columns with dtype: `int64`: []


# 3. Data selection

In pandas.DataFrame you could select

  Row/s by position (integer number [0 .. number of rows - 1]) .iloc or by DataFrame.index .loc:   

```
  data.loc[0]  
  data.loc[5:10]  
  data.iloc[0]  
  data.iloc[5:10]   
```

Though, this is probably the worst way to manipulate rows.   
  Columns by name

```
  data[columname]
```

  Row/s and columns

```
  data.loc[10, columname]  
  data.iloc[10, columname]  
```

Using boolean mask

```
  mask = data[columname] > value  
  data[mask]  
```

You could combine multiple conditions using & or | (and, or)   

```
cond1 = data[columname1] > value1  
cond2 = data[columname2] > value2  
data[cond1 & cond2]  
```

Using queries .query():  

```
value = 5 
data.query("columname > value")  
```

You could combine multiple conditions using and, or  

```
data.query("(columname1 > value1) and (columname2 > value2)")
```

and others. See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html for more examples.

Remember to use different quotation marks " or ' for columnname inside a query.


In [15]:
# Select rows by position (int) 

# Q8.1 What is the temperature of the time slot with index 777?
print(data.loc[777, 'temp'])

# Q8.2 What is the weather description of the time slot with index 999?
print(data.loc[999, 'weather_description'])

# Q8.3 How much is cloud coverage with index 1337?
print(data.loc[1337, 'clouds_all'])

# Q8.4 What is the weather main of the time slot with index 314?
print(data.loc[314, 'weather_main'])

# Q8.5 When was the time slot with index of 2718 observed?
print(pd.to_datetime(data.loc[2718, 'date_time']).time(), 'FULL:', data.loc[2718, 'date_time'])


278.5
overcast clouds
1.0
Clouds
14:00:00 FULL: 2013-01-06 14:00:00


In [16]:
# Select rows by index (int)

# Q9.1 What is the temperature of the time slot on index 1102?
print(data.iloc[1102]['temp'], ':',
      data.iloc[1102, data.columns.get_loc('temp')])

# Q9.2 What is the weather description of the time slot on index 5695?
print(data.iloc[5695]['weather_description'], ':',
      data.iloc[5695, data.columns.get_loc('weather_description')])

# Q9.3 How much is cloud coverage on the index 1045?
print(data.iloc[1045]['clouds_all'], ':',
      data.iloc[1045, data.columns.get_loc('clouds_all')]) 

# Q9.4 What is the weather main of the time slot from index 252?
print(data.iloc[252]['weather_main'], ':',
      data.iloc[252, data.columns.get_loc('weather_main')])

# Q9.5 When was the time slot with index of 38 captured?
print(data.iloc[38]['date_time'], ':',
      data.iloc[38, data.columns.get_loc('date_time')])

280.43 : 280.43
overcast clouds : overcast clouds
20.0 : 20.0
Clear : Clear
2012-10-04 03:00:00 : 2012-10-04 03:00:00


In [17]:
data.columns.get_loc('clouds_all')

4

In [18]:
# Using mask or .query syntax select rows/columns (int)

# Q10.1 How many time slots have less than 270 temperature?
print(data[data['temp'] < 270].shape[0], ':',
      data.query("temp < 270").shape[0])

# Q10.2 When was the first "light intensity drizzle" in weather description captured?
print(data[data['weather_description'] == 'light intensity drizzle']['date_time'].iloc[0], ':',
      data.query("weather_description == 'light intensity drizzle'")['date_time'].iloc[0])

# Q10.3 How many time slots have cloud coverage more than 75?
print(data[data['clouds_all'] > 75].shape[0], ':',
      data.query("clouds_all > 75").shape[0])

# Q10.4 How many time slots are foggy? (weather_main = Fog)
print(data[data['weather_main'] == 'Fog'].shape[0], ':',
      data.query("weather_main == 'Fog'").shape[0])

# Q10.5 When was the last observed timeslot with weather_description "heavy snow"?
print(data[data['weather_description'] == 'heavy snow']['date_time'].iloc[-1], ':',
      data.query("weather_description == 'heavy snow'")['date_time'].iloc[-1])

9308 : 9308
2012-10-10 07:00:00 : 2012-10-10 07:00:00
18036 : 18036
912 : 912
2018-04-15 18:00:00 : 2018-04-15 18:00:00


In [19]:
# Q11.1 What is the traffic volume of November 20th 2016, at 20:00?
print(data.query('date_time == "2016-11-20 20:00:00"')['traffic_volume'].iloc[0])

# Q11.2 What is the amount of rain in the 70th rainy time slot (non-zero rain) of the dataset?
print(data.query('rain_1h != 0')['rain_1h'].iloc[70])

# Q11.3 How much cloud coverage percentage were in sky on October 16th 2012 at 19:00?
print(data.query('date_time == "2012-10-16 19:00:00"')['clouds_all'].iloc[0])

# Q11.4 What is the `traffic_volume` of a thirty fourth sample with `clouds_all` == 90?
print(data.query('clouds_all == 90')['traffic_volume'].iloc[34])

# Q11.5 What is the "weather_description" in the 20th "weather_main" with Thunderstorm?
print(data.query('weather_main == "Thunderstorm"')['weather_description'].iloc[20])

2070.0
0.51
68.0
3833.0
proximity thunderstorm with rain


In [20]:
# Q12.1 What is the temperature of the tenth holiday?
print(data[data['holiday'] != 'None'].iloc[8]['temp'])

# Q12.2 What is the traffic volume for 99-th time slot with cloud coverage 75 percent?
print(data[data['clouds_all'] == 75].iloc[98]['traffic_volume'])

# Q12.3 How much is the temperature of the twelfth holiday?
print(data[data['holiday'] != 'None'].iloc[11]['temp'])

# Q12.4 How much is the temperature the 666-th time slot with weather_description 'proximity thunderstorm'?
print(data[data['weather_description'] != 'proximity thunderstorm'].iloc[665]['temp'])

# Q12.5 What is the temperature of 1337-th time slot with clear sky (clouds_all <= 20)?
print(data[data['clouds_all'] <= 20].iloc[1337]['temp'])

297.42
1437.0
275.44
275.03
277.06


# 4. Creating new columns

Creating new column of pandas.DataFrame is as easy as:
```
data['new_awesome_column'] = [] 
```
that's it. But such a column is relatively useless. Typically, you would compute something new based on existing data and save it in a new column. For example one might want to sum a number of existing columns:
```
data['sum'] = data[col1] + data[col2] + ...
```
Pandas also provides another powerfull tool: .apply, .map(), .applymap() methods (they are kinda the same, but not quite). https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas . They allow you to apply some function to every value in the column/s (row-wise) or row (column-wise) or cell (element-wise). For example, same computations of sum using .apply():
```
data['sum'] = data[[col1, col2, col3]].apply(sum, axis=1)
```
you are not restricted to existent functions, .apply() accepts any function (including lambda functions):
```
data['sum'] = data[[col1, col2, col3]].apply(lambda x: x[0]+x[1]+x[2], axis=1)
```
or ordinary python function (if this it should have complex behaviour):
```
def _sum(x):
    total = 0
    for elem in x:
        total += elem
    return total

data['sum'] = data[[col1, col2, col3]].apply(_sum, axis=1) 
```
Many pandas methods has axis parameter axis=0 refers to rows, axis=1 refers to columns.

Warning. You should never use for loops to sum numerical elements from the container.

In [21]:
# Create new columns using the old ones (new column in your DataFrame)

# Q13.1 Create a `temp_in_celcius` column from the existing `temp` (kelvin) using any method above
data['temp_in_celcius'] = data['temp'] - 273

# Q13.2 Create a new bool column `hot` which indicates whether the time slot was hot (temp > 300)
data['hot'] = data['temp'] > 300

# Q13.3 Create a new bool column `rainy_and_cloudy` which indicates whether it was rainy (>0.1) AND cloudy (>50)
data['rainy_and_cloudy'] = (data['rain_1h'] > 0.1)&(data['clouds_all'] > 50)

# Q13.4 Create a new bool column `is_holiday` which indicates whether the day of the time slot falls on any holiday
data['is_holiday'] = data['holiday'] != 'None'

# Q13.5 Create a new column `traffic_cat` by splitting a `traffic_volume` into 5 ([1..5]) distinct intervals: 0 < x <=20%,
# 20% < x <= 40%, ... 80% < x <= 100% percentiles. You could use `.quantile()` to compute percentiles.
data['traffic_cat'] = pd.qcut(data['traffic_volume'], q=np.arange(0, 1.2, 0.2), labels=range(1, 6))

In [22]:
# Using mask or .query syntax select rows/columns (int).
# For working with dates, define helper functions that operate on the date_time string.
def season(date_time_str):
    month = date_time_str.split('-')[1]
    if month in ['09', '10', '11']:
        return 'autumn'
    elif month in  ['06', '07', '08']:
        return 'summer'
    elif month in ['03', '04', '05']:
        return 'spring'
    elif month in ['12', '01', '02']:
        return 'winter'
    else:
        raise ValueError(month)

data['season'] = data['date_time'].apply(season)


# Q14.1 How many cloudy time slots were captured in autumn 2016? Including both start and end day.
print(data[(data['weather_main'] == 'Clouds')&
           (data['date_time'].str.startswith('2016'))&
           (data['season'] == 'autumn')].shape[0])

print(data[(data['weather_main'] == 'Clouds')&
           (data['date_time'].str.startswith('2016'))&
           (data['date_time'].apply(lambda x: x.split('-')[1]).isin(['09', '10', '11']))].shape[0])
print('------')

# Q14.2 How many rainy time slots that were captured in the fall, with traffic volume more than 2000?
print(data[(data['weather_main'] == 'Rain')&
           (data['season'] == 'autumn')&
           (data['traffic_volume'] > 2000)].shape[0])

print(data[(data['weather_main'] == 'Rain')&
           (data['traffic_volume'] > 2000)&
           (data['date_time'].apply(lambda x: x.split('-')[1]).isin(['09', '10', '11']))].shape[0])
print('------')

# Q14.3 How many time slots that are warmer than 270, have weather main "Clouds"?
print(data[(data['weather_main'] == 'Clouds')&
           (data['temp'] > 270)].shape[0])

# Q14.4 What is the minimum traffic volume of time slots captured on March 8th (all years), that was warmer than 290?
print(data[(data['date_time'].str.contains('03-08'))&
           (data['temp'] > 290)]['traffic_volume'].min())

# Q14.5 How much is the maximum traffic volume for the time slots were captured in June 2017 and has clear sky (weather_main)?
print(data[(data['date_time'].str.startswith('2017-06'))&
           (data['weather_main'] == 'Clear')]['traffic_volume'].max())

del data['season']

643
643
------
978
978
------
12133
4780.0
6673.0


In [23]:
# Using mask or .query syntax select rows/columns and compute simple statistics (float)

# Q15.1 What was the average temperature of time slots with main weather "Haze"?
print(data[data['weather_main'] == 'Haze']['temp'].mean().round(3))

# Q15.2 What was the traffic volume of the coldest time slot of the year 2016?
print(data.iloc[data[data['date_time'].str.startswith('2016')]['temp'].argmin()]['traffic_volume'])

# Q15.3 What was the traffic volume of the highest amount of snow in one hour?
print(data.iloc[data['snow_1h'].argmax()]['traffic_volume'])

# Q15.4 What is the median of temperatures captured in April 2017?
print(data[data['date_time'].str.startswith('2017-04')]['temp'].median())

# Q15.5 What is the maximum temperature of time slots with clear sky?
print(data[data['weather_main'] == 'Clear']['temp'].max())

275.805
383.0
5167.0
282.01
308.43


In [24]:
# Using mask or .query syntax select rows/columns (float)

# Q16.1 What is the average temperature in celcius of the time slots with rainy_and_coudy=True?
print(data[data['rainy_and_cloudy']]['temp_in_celcius'].mean().round(3))

# Q16.2 What is the average traffic volume on holidays?
print(data[data['is_holiday']]['traffic_volume'].mean().round(3))

# Q16.3 What is the average traffic volume on non-holidays?
print(data[~data['is_holiday']]['traffic_volume'].mean().round(3))

# Q16.4 What is the average traffic volume in the highest quantile?
print(data[data['traffic_cat'] == max(data['traffic_cat'])]['traffic_volume'].mean().round(3))

# Q16.5 What is the average traffic volume in the lowest quantile?
print(data[data['traffic_cat'] == min(data['traffic_cat'])]['traffic_volume'].mean().round(3))


13.735
865.443
3262.894
5870.913
485.554


# 5. Basic date processing

You figure out that column date is to harsh for you, so you decided to convert it to a more plausible format:

- Use pandas method to_datetime() to convert the date to a good format.
- Extract year, month, day and weekday from your new date column. Save them to separate columns.
- How many columns has your data now?
- Drop column date, remember to set inplace parameter to True.

Hint: for datetime formatted date you could extract the year as follow:
```
data.date.dt.year
```
Very often date could be a ridiculously rich feature, sometimes it is holidays that matters, sometimes weekends, sometimes some special days like black friday.

Learn how to work with date in Python!


In [25]:
# Create new columns based on `Captured` column

# Q17.1 Convert date to datetime format
data['date_time'] = pd.to_datetime(data['date_time'])

# Q17.2 Extract and store `year`
data['year'] = data['date_time'].dt.year

# Q17.3 Extract and store `month`
data['month'] = data['date_time'].dt.month

# Q17.4 Extract and store `day`
data['day'] = data['date_time'].dt.day

# Q17.5 Extract and store `weekday` (Monday - 0, Sunday - 6)
data['weekday'] = data['date_time'].dt.weekday

# Q17.6 Extract and store `hour`
data['hour'] = data['date_time'].dt.hour

In [26]:
# Find some date related information from the data (int)

# Q18.1 What is the weekday with the highest traffic volume?
print(data.iloc[data['traffic_volume'].argmax()]['weekday'])

# Q18.2 What is the weekday with the lowest traffic volume?
print(data.iloc[data['traffic_volume'].argmin()]['weekday'])

# Q18.3 What is the average traffic volume during months of September?
print(data[data['month'] == 9]['traffic_volume'].mean().round(3))

# Q18.4 What is the average traffic volume in the time period between 15-19 hours
print(data[data['hour'].isin(range(15, 20))]['traffic_volume'].mean().round(3))

# Q18.5 What is the average traffic volume on World Bicycle Day (June 3)?
print(data[(data['month'] == 6)&(data['day'] == 3)]['traffic_volume'].mean().round(3))


3
5
3303.049
4749.296
3445.976


# 6. Groupby

from the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria.
- Applying a function to each group independently.
- Combining the results into a data structure.

`.groupby()` is one of the most powerfull tool for feature engineering. Very often it is used to group object with the same categorical characteristics and compute some statistics (e.g. mean, max, etc.) of a their numerical characteric.

Instead of computing average traffic volume with for each month you could compute average traffic volumes for every month in a single command:
```
data.groupby('month')['traffic_volume'].mean()
```
You could also make multi-column groups:
```
data.groupby(['weekday','month'])['traffic_volume'].min()
```
next, you could compute multiple aggregation functions:
```
data.groupby(['weekday','month'])['traffic_volume'].agg([min, max])
```
instead of using built-in functions you could compute custom functions using apply:
```
import numpy as np
data.groupby(['weekday','month'])['traffic_volume'].apply(lambda x: np.quantile(x, .5))
```
and the coolest thing now is that you can map the results of groupby back on your DataFrame!
```
gp = data.groupby(['month'])['traffic_volume'].median()
data['gp_feature'] = data['month'].map(gp)
```
Now, if some timeslot has month == 2, its gp_feature will be equal to the median traffic volume amongst all observations in February

Read more examples in the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html


In [27]:
# Create some groupby features

# Q19.1 `traffic_by_year` groupby `year` and compute median traffic volume.
data['traffic_by_year'] = data['year'].map(data.groupby('year')['traffic_volume'].median())

# Q19.2 `traffic_by_weekday` groupby `weekday` and compute median traffic volume.
data['traffic_by_weekday'] = data['weekday'].map(data.groupby('weekday')['traffic_volume'].median())

# Q19.3 `temperature_by_traffic` groupby `traffic_cat` and compute average temperature in celsius.
data['temperature_by_traffic'] = data['traffic_cat'].map(data.groupby('traffic_cat')['temp_in_celcius'].mean())


# Make sure your .ipynb is linearly executable     
# Kernel -> Restart & Run All -> No ERROR cells

In [28]:
from DataAccessor import DataAccessor
from Features import Features

data = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')
data.dropna(inplace=True)
data_wrapped = DataAccessor(data)

main_features = Features.compute(data_wrapped) #Features.compute(data_wrapped, include_initial=True)
print(main_features.shape)

(48190, 15)


In [29]:
main_features.sample(10)

,precipitation_total,temp_in_celcius,hot,cold,rainy_and_cloudy,is_holiday,year,month,day,weekday,hour,is_day_off,season,weather,traffic_cat
47107,1.78,24.57,False,False,False,False,2018,8,26,6,22,True,summer,Rain,1
12121,0.00,-17.16,False,True,False,False,2014,2,9,6,10,True,winter,Clear,2
17556,0.00,16.34,False,False,False,False,2015,8,19,2,14,False,summer,Mist,4
43935,0.00,12.82,False,False,False,False,2018,5,9,2,5,False,spring,Rain,2
28385,0.00,1.52,False,False,False,False,2016,11,23,2,12,False,autumn,Rain,3
42321,0.00,-3.18,False,False,False,False,2018,3,14,2,1,False,spring,Clear,0
18564,0.00,19.51,False,False,False,False,2015,9,24,3,14,False,autumn,Drizzle,4
5928,0.00,8.10,False,False,False,False,2013,5,10,4,21,False,spring,Clouds,2
2229,0.00,-1.66,False,False,False,False,2012,12,20,3,5,False,winter,Clouds,1
35614,0.00,16.56,False,False,False,False,2017,7,24,0,1,False,summer,Clouds,0


In [30]:
# from catboost import CatBoostClassifier, Pool

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

# train_x, test_x, train_y, test_y = train_test_split(main_features.drop(columns='traffic_cat'), 
#                                                     main_features['traffic_cat'],
#                                                     test_size=0.25, 
#                                                     stratify=main_features['traffic_cat'])

# print(train_x.shape, test_x.shape)
# print(train_y.shape, test_y.shape)

(36142, 14) (12048, 14)
(36142,) (12048,)


In [31]:
# cat_features = [
#     'hot', 
#     'cold', 
#     'rainy_and_cloudy',
#     'is_holiday', 
#     'is_day_off', 
#     'season', 
#     'weather'
# ]

# train_pool = Pool(data=train_x, label=train_y, cat_features=cat_features)
# test_pool = Pool(data=test_x, label=test_y, cat_features=cat_features)

In [32]:
# clf = CatBoostClassifier(
#     depth=2,
#     iterations=150,
#     learning_rate=0.05,
#     bootstrap_type='Bayesian',
#     bagging_temperature=2,
#     leaf_estimation_method='Newton',
#     boosting_type='Ordered',
#     logging_level='Silent',
# )

# clf.fit(train_pool)

In [33]:
# print(accuracy_score(test_y, clf.predict(test_pool)))
# print(f1_score(test_y, clf.predict(test_pool), average='macro'))
# print(roc_auc_score(test_y, clf.predict_proba(test_pool), multi_class='ovo', average='macro'))

0.7664342629482072
0.7600764701038905
0.9417260848244796
